In [29]:
data_dir = "/home/pneekhara/Datasets/78419/Hi_Fi_TTS_v_0_backup"
target_dir = "/home/pneekhara/filelists"

In [30]:
import random
import os
import json


def make_universal_hifigan_filelist(speaker_id_mapping, split):
    
    all_records = []
    for speaker_id in speaker_id_mapping:
        clean_other = speaker_id_mapping[speaker_id]
        file_list_name = "{}_manifest_{}_{}.json".format(speaker_id, clean_other, split)
        with open(os.path.join(data_dir, file_list_name), 'r') as f:
            speaker_records = [json.loads(l) for l in f.read().split("\n") if len(l) > 0]
            print ("Num Records : {} for speaker {}".format(len(speaker_records), speaker_id))
        
        for row in speaker_records:
            all_records.append(row)
    random.seed(0)
    random.shuffle(all_records)
    if split == "dev":
        all_records = all_records[:150]
    else:
        all_records = all_records[:150000]
        
    target_fp = os.path.join(target_dir, "hifigan_mixed_{}.json".format(split))
    with open(target_fp, 'w') as f:
        for record in json.loads(json.dumps(all_records)):
            record['audio_filepath'] = record['audio_filepath'][record['audio_filepath'].find("wav/"):]
            f.write(json.dumps(record) + "\n")
    
    target_fp = os.path.join(target_dir, "hifigan_mixed_{}_ngc.json".format(split))
    with open(target_fp, 'w') as f:
        for record in json.loads(json.dumps(all_records)):
            record['audio_filepath'] = record['audio_filepath'][record['audio_filepath'].find("wav/"):]
            record['audio_filepath'] = "/raid/HiFiDataset/Hi_Fi_TTS_v_0/" + record['audio_filepath']
            f.write(json.dumps(record) + "\n")
    
    target_fp = os.path.join(target_dir, "hifigan_mixed_{}_local.json".format(split))
    with open(target_fp, 'w') as f:
        for record in json.loads(json.dumps(all_records)):
            record['audio_filepath'] = record['audio_filepath'][record['audio_filepath'].find("wav/"):]
            record['audio_filepath'] = "/home/pneekhara/Datasets/78419/Hi_Fi_TTS_v_0_backup/" + record['audio_filepath']
            f.write(json.dumps(record) + "\n")
    
def make_sub_file_list(speaker_id, clean_other, split, num_samples, total_duration, seed=42):
    file_list_name = "{}_manifest_{}_{}.json".format(speaker_id, clean_other, split)
    with open(os.path.join(data_dir, file_list_name), 'r') as f:
        all_records = [json.loads(l) for l in f.read().split("\n") if len(l) > 0]
    for r in all_records:
        r['audio_filepath'] = r['audio_filepath'][r['audio_filepath'].find("wav/"):]
    random.seed(seed)
    random.shuffle(all_records)
    
    if num_samples is not None and total_duration is None:
        sub_records = all_records[:num_samples]
        fname_extension = "ns_{}".format(num_samples)
    elif num_samples is None and total_duration is not None:
        sub_record_duration = 0.0
        sub_records = []
        for r in all_records:
            sub_record_duration += r['duration']
            if sub_record_duration > total_duration:
                print ("Sub record duration reached {}".format(total_duration), len(sub_records))
                break
            sub_records.append(r)
        fname_extension = "dur_{}_mins".format( int(round(total_duration/60.0 )))
    elif num_samples is None and total_duration is None:
        sub_records = all_records
        fname_extension = "ns_all"
    else:
        raise NotImplementedError()
    print ("num subrecs", len(sub_records))
    target_fp = os.path.join(target_dir, "{}_mainifest_{}_{}.json".format(speaker_id, split, fname_extension))
    with open(target_fp, 'w') as f:
        for record in json.loads(json.dumps(sub_records)):
            f.write(json.dumps(record) + "\n")
    
    target_fp = os.path.join(target_dir, "{}_mainifest_{}_{}_ngc.json".format(speaker_id, split, fname_extension))
    with open(target_fp, 'w') as f:
        for record in json.loads(json.dumps(sub_records)):
            record['audio_filepath'] = record['audio_filepath'][record['audio_filepath'].find("wav/"):]
            record['audio_filepath'] = "/raid/HiFiDataset/Hi_Fi_TTS_v_0/" + record['audio_filepath']
            f.write(json.dumps(record) + "\n")
    
    target_fp = os.path.join(target_dir, "{}_mainifest_{}_{}_local.json".format(speaker_id, split,  fname_extension))
    with open(target_fp, 'w') as f:
        for record in json.loads(json.dumps(sub_records)):
            record['audio_filepath'] = record['audio_filepath'][record['audio_filepath'].find("wav/"):]
            record['audio_filepath'] = "/home/pneekhara/Datasets/78419/Hi_Fi_TTS_v_0_backup/" + record['audio_filepath']
            f.write(json.dumps(record) + "\n")

def mix_file_list(speaker_id, clean_other, split, num_samples, total_duration, original_speaker_id, original_clean_other, n_orig=None, seed=42):
    file_list_name = "{}_manifest_{}_{}.json".format(speaker_id, clean_other, split)
    with open(os.path.join(data_dir, file_list_name), 'r') as f:
        all_records = [json.loads(l) for l in f.read().split("\n") if len(l) > 0]
    for r in all_records:
        r['audio_filepath'] = r['audio_filepath'][r['audio_filepath'].find("wav/"):]
    
    original_file_list_name = "{}_manifest_{}_{}.json".format(original_speaker_id, original_clean_other, "train")
    with open(os.path.join(data_dir, original_file_list_name), 'r') as f:
        original_all_records = [json.loads(l) for l in f.read().split("\n") if len(l) > 0]
    for r in original_all_records:
        r['audio_filepath'] = r['audio_filepath'][r['audio_filepath'].find("wav/"):]
    
    random.seed(seed)
    if n_orig is not None:
        random.shuffle(original_all_records)
        original_all_records = original_all_records[:n_orig]
        
    random.seed(seed)
    random.shuffle(all_records)
    
    if num_samples is not None and total_duration is None:
        sub_records = all_records[:num_samples]
        fname_extension = "ns_{}".format(num_samples)
    elif num_samples is None and total_duration is not None:
        sub_record_duration = 0.0
        sub_records = []
        for r in all_records:
            sub_record_duration += r['duration']
            if sub_record_duration > total_duration:
                print ("Sub record duration reached {}".format(total_duration), len(sub_records))
                break
            sub_records.append(r)
        fname_extension = "dur_{}_mins".format( int(round(total_duration/60.0 )))
    elif num_samples is None and total_duration is None:
        sub_records = all_records
        fname_extension = "ns_all"
    else:
        raise NotImplementedError()
        
    print(len(original_all_records))
    sub_records1 = json.loads(json.dumps(sub_records))
    sub_records2 = json.loads(json.dumps(sub_records))
    sub_records3 = json.loads(json.dumps(sub_records))
    target_fp = os.path.join(target_dir, "{}_mainifest_{}_{}_mix_{}.json".format(speaker_id, split, fname_extension, original_speaker_id))
    with open(target_fp, 'w') as f:
        for ridx, original_record in enumerate(original_all_records):
            new_speaker_record = sub_records1[ridx % len(sub_records1)]
            new_speaker_record['speaker'] = 1
            original_record['speaker'] = 0
            f.write(json.dumps(original_record) + "\n")
            f.write(json.dumps(new_speaker_record) + "\n")
    
    target_fp = os.path.join(target_dir, "{}_mainifest_{}_{}_ngc_mix_{}.json".format(speaker_id, split, fname_extension, original_speaker_id))
    with open(target_fp, 'w') as f:
        for ridx, original_record in enumerate(original_all_records):
            original_record['audio_filepath'] = original_record['audio_filepath'][original_record['audio_filepath'].find("wav/"):]
            original_record['audio_filepath'] = "/raid/HiFiDataset/Hi_Fi_TTS_v_0/" + original_record['audio_filepath']
            
            new_speaker_record = sub_records2[ridx % len(sub_records2)]
            new_speaker_record['audio_filepath'] = new_speaker_record['audio_filepath'][new_speaker_record['audio_filepath'].find("wav/"):]
            new_speaker_record['audio_filepath'] = "/raid/HiFiDataset/Hi_Fi_TTS_v_0/" + new_speaker_record['audio_filepath']
            
            new_speaker_record['speaker'] = 1
            original_record['speaker'] = 0
            f.write(json.dumps(original_record) + "\n")
            f.write(json.dumps(new_speaker_record) + "\n")
    
    target_fp = os.path.join(target_dir, "{}_mainifest_{}_{}_local_mix_{}.json".format(speaker_id, split,  fname_extension, original_speaker_id))
    with open(target_fp, 'w') as f:
        for ridx, original_record in enumerate(original_all_records):
            original_record['audio_filepath'] = original_record['audio_filepath'][original_record['audio_filepath'].find("wav/"):]
            original_record['audio_filepath'] = "/home/pneekhara/Datasets/78419/Hi_Fi_TTS_v_0_backup/" + original_record['audio_filepath']
            
            new_speaker_record = sub_records3[ridx % len(sub_records3)]
            new_speaker_record['audio_filepath'] = new_speaker_record['audio_filepath'][new_speaker_record['audio_filepath'].find("wav/"):]
            new_speaker_record['audio_filepath'] = "/home/pneekhara/Datasets/78419/Hi_Fi_TTS_v_0_backup/" + new_speaker_record['audio_filepath']
            
            new_speaker_record['speaker'] = 1
            original_record['speaker'] = 0
            f.write(json.dumps(original_record) + "\n")
            f.write(json.dumps(new_speaker_record) + "\n")

In [31]:
speaker_id_mapping = {
#     92 : 'clean',
#     6097 : 'clean',
    8051 : 'clean',
    11697 : 'other',
    6670 : 'other',
    6671 : 'other',
    9017 : 'clean',
    11614 : 'other',
    9136 : 'other',
    12787 : 'other'
}
# make_universal_hifigan_filelist(speaker_id_mapping, 'dev')

In [32]:
# make_sub_file_list(6097, "clean", "train", None, 1*60)
# make_sub_file_list(6097, "clean", "train", None, 5*60)
# make_sub_file_list(6097, "clean", "train", None, 30*60)
# make_sub_file_list(6097, "clean", "train", None, 60*60)
for speaker in speaker_id_mapping:
    make_sub_file_list(speaker, speaker_id_mapping[speaker], "dev", None, None)

num subrecs 50
num subrecs 50
num subrecs 50
num subrecs 50
num subrecs 50
num subrecs 50
num subrecs 50
num subrecs 50


In [12]:
mix_file_list(6097, "clean", "train", None, 1*60, 8051, "clean", n_orig=5000)
mix_file_list(6097, "clean", "train", None, 5*60, 8051, "clean", n_orig=5000)
mix_file_list(6097, "clean", "train", None, 30*60, 8051, "clean", n_orig=5000)
mix_file_list(6097, "clean", "train", None, 60*60, 8051, "clean", n_orig=5000)

Sub record duration reached 60 17
5000
Sub record duration reached 300 108
5000
Sub record duration reached 1800 656
5000
Sub record duration reached 3600 1319
5000


In [10]:
def get_best_ckpt(new_speaker_id, duration_mins, mixing_enabled, original_speaker_id):
    if not mixing_enabled:
        exp_dir = "/home/pneekhara/ExperimentsAutomated/{}_to_{}_no_mixing_{}_mins".format(original_speaker_id, new_speaker_id, duration_mins)
    else:
        exp_dir = "/home/pneekhara/ExperimentsAutomated/{}_to_{}_mixing_{}_mins".format(original_speaker_id, new_speaker_id, duration_mins)
    
    ckpt_candidates = []
    for root, dirs, files in os.walk(exp_dir):
        for file in files:
            if file.endswith(".ckpt"):
                val_error = float(file.split("v_loss=")[1].split("-epoch")[0])
                ckpt_candidates.append( (val_error, os.path.join(root, file)))
    ckpt_candidates.sort()
    
    return ckpt_candidates
    

In [27]:
pitch_stats = {
    92 : {
        'mean' : 214.5,
        'std' : 30.9,
        'fmin' : 80,
        'fmax' : 512
    },
    6097 : {
        'mean' : 121.9 ,
        'std' : 23.1,
        'fmin' : 30,
        'fmax' : 512
    }
}
def generate_training_command_old(new_speaker_id, duration_mins, mixing_enabled, original_speaker_id, use_new_pitch_stats=False,
                             ckpt = "/home/pneekhara/PreTrainedModels/FastPitch.nemo"):
    
    def _find_epochs(duration_mins, mixing_enabled, n_orig=None):
        epochs = 200 # default.
        if duration_mins == 5:
            epochs = 1000
        elif duration_mins == 30:
            epochs = 300
        elif duration_mins == 60:
            epochs = 150
        
        if mixing_enabled:
            if duration_mins == 5:
                epochs = epochs/50 + 1
            elif duration_mins == 30:
                epochs = epochs/10 + 1
            elif duration_mins == 60:
                epochs = epochs/5 + 1
        
        return int(epochs)
            
            
    if ckpt.endswith(".nemo"):
        ckpt_arg_name = "init_from_nemo_model"
    else:
        ckpt_arg_name = "init_from_ptl_ckpt"
    if not mixing_enabled:
        config_name = "fastpitch_align_finetuningnomixing"
        train_dataset = "../filelists/{}_mainifest_train_dur_{}_mins_local.json".format(new_speaker_id, duration_mins)
        val_dataset = "../filelists/{}_mainifest_dev_ns_all_local.json".format(new_speaker_id)
        prior_folder = "/home/pneekhara/Datasets/Priors{}".format(new_speaker_id)
        exp_dir = "/home/pneekhara/ExperimentsAutomatedResetPitch/{}_to_{}_no_mixing_{}_mins".format(original_speaker_id, new_speaker_id, duration_mins)
    else:
        config_name = "fastpitch_align_finetuningmixing"
        train_dataset = "../filelists/{}_mainifest_train_dur_{}_mins_local_mix_{}.json".format(new_speaker_id, duration_mins, original_speaker_id)
        val_dataset = "../filelists/{}_mainifest_dev_ns_all_local.json".format(new_speaker_id)
        prior_folder = "/home/pneekhara/Datasets/Priors_{}_mix_{}".format(new_speaker_id, original_speaker_id)
        exp_dir = "/home/pneekhara/ExperimentsAutomatedResetPitch/{}_to_{}_mixing_{}_mins".format(original_speaker_id, new_speaker_id, duration_mins)
    max_epochs = _find_epochs(duration_mins, mixing_enabled, n_orig=None)
    
    training_command = "python examples/tts/fastpitch2_finetune.py --config-name={} train_dataset={} validation_datasets={} +{}={} trainer.max_epochs={} trainer.check_val_every_n_epoch=1 prior_folder={} model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 exp_manager.exp_dir={}".format(
        config_name, train_dataset, val_dataset, ckpt_arg_name, ckpt, max_epochs, prior_folder, exp_dir)
    if use_new_pitch_stats:
        training_command += " model.pitch_avg={} model.pitch_std={} model.pitch_fmin={} model.pitch_fmax={}".format(
            pitch_stats[new_speaker_id]['mean'], 
            pitch_stats[new_speaker_id]['std'],
            pitch_stats[new_speaker_id]['fmin'],
            pitch_stats[new_speaker_id]['fmax']
        )
        
    return training_command


data_dir = "/home/pneekhara/Datasets/78419/Hi_Fi_TTS_v_0_backup"
filelist_dir = "/home/pneekhara/filelists"
exp_base_dir = "/home/pneekhara/ExperimentsAutomatedResetPitch"
exp_base_dir_hifi = "/home/pneekhara/ExperimentsHiFiFinetuning"

def generate_training_command_new(new_speaker_id, duration_mins, mixing_enabled, original_speaker_id, ckpt="/home/pneekhara/PreTrainedModels/FastPitch.nemo", use_new_pitch_stats=False):
    """
    Generates the training command string to be run from the NeMo/ directory. Assumes we have created the finetuning filelists
    using the instructions given above.
    
    Arguments:
    new_speaker_id -- speaker id of the new HiFiTTS speaker
    duration_mins -- total minutes of the new speaker data (same as that used for creating the filelists)
    mixing_enabled -- True or False depending on whether we want to mix the original speaker data or not
    original_speaker_id -- speaker id of the original HiFiTTS speaker
    use_new_pitch_stats -- whether to use pitch_stats dictionary given above or not
    ckpt: Path to pretrained FastPitch checkpoint
    
    Returns:
    Training command string
    """
    def _find_epochs(duration_mins, mixing_enabled, n_orig=None):
        # estimated num of epochs 
        if duration_mins == 1:
            epochs = 5000
        elif duration_mins == 5:
            epochs = 1000
        elif duration_mins == 30:
            epochs = 300
        elif duration_mins == 60:
            epochs = 150
        
        if mixing_enabled:
            if duration_mins == 1:
                epochs = epochs/70 + 1
            elif duration_mins == 5:
                epochs = epochs/50 + 1
            elif duration_mins == 30:
                epochs = epochs/10 + 1
            elif duration_mins == 60:
                epochs = epochs/5 + 1
        
        return int(epochs)
            
            
    if ckpt.endswith(".nemo"):
        ckpt_arg_name = "init_from_nemo_model"
    else:
        ckpt_arg_name = "init_from_ptl_ckpt"
    if not mixing_enabled:
        train_dataset = "{}_mainifest_train_dur_{}_mins_local.json".format(new_speaker_id, duration_mins)
        val_dataset = "{}_mainifest_dev_ns_all_local.json".format(new_speaker_id)
        prior_folder = os.path.join(data_dir, "Priors{}".format(new_speaker_id))
        exp_dir = "{}_to_{}_no_mixing_{}_mins".format(original_speaker_id, new_speaker_id, duration_mins)
        n_speakers = 1
    else:
        train_dataset = "{}_mainifest_train_dur_{}_mins_local_mix_{}.json".format(new_speaker_id, duration_mins, original_speaker_id)
        val_dataset = "{}_mainifest_dev_ns_all_local.json".format(new_speaker_id)
        prior_folder = os.path.join(data_dir, "Priors_{}_mix_{}".format(new_speaker_id, original_speaker_id))
        exp_dir = "{}_to_{}_mixing_{}_mins".format(original_speaker_id, new_speaker_id, duration_mins)
        n_speakers = 2
    train_dataset = os.path.join(filelist_dir, train_dataset)
    val_dataset = os.path.join(filelist_dir, val_dataset)
    exp_dir = os.path.join(exp_base_dir, exp_dir)
                                    
    max_epochs = _find_epochs(duration_mins, mixing_enabled, n_orig=None)
    config_name = "fastpitch_align_44100.yaml"
    
    training_command = "python examples/tts/fastpitch2_finetune.py --config-name={} train_dataset={} validation_datasets={} +{}={} trainer.max_epochs={} trainer.check_val_every_n_epoch=1 prior_folder={} model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 exp_manager.exp_dir={} model.n_speakers={}".format(
        config_name, train_dataset, val_dataset, ckpt_arg_name, ckpt, max_epochs, prior_folder, exp_dir, n_speakers)
    if use_new_pitch_stats:
        training_command += " model.pitch_avg={} model.pitch_std={} model.pitch_fmin={} model.pitch_fmax={}".format(
            pitch_stats[new_speaker_id]['mean'], 
            pitch_stats[new_speaker_id]['std'],
            pitch_stats[new_speaker_id]['fmin'],
            pitch_stats[new_speaker_id]['fmax']
        )
    training_command += " model.optim.lr=2e-4 ~model.optim.sched"
    return training_command

def generate_hifigan_training_command(new_speaker_id, duration_mins, mixing, original_speaker_id=8051, hifipretrained_ckpt="/home/pneekhara/PreTrainedModels/HiFiMix_No_92_6097.ckpt"):
    
    
    if not mixing:
        exp_dir = "{}_to_{}_no_mixing_{}_mins".format(original_speaker_id, new_speaker_id, duration_mins)
        train_manifest_file = "{}_mainifest_train_dur_{}_mins_local.json".format(new_speaker_id, duration_mins)
        trainer_max_steps = 2000
    else:
        exp_dir = "{}_to_{}_mixing_{}_mins".format(original_speaker_id, new_speaker_id, duration_mins)
        train_manifest_file = "{}_mainifest_train_dur_{}_mins_local_mix_{}.json".format(new_speaker_id, duration_mins, original_speaker_id)
        trainer_max_steps = 4000
        
    val_manifest_file = "{}_mainifest_dev_ns_all_local.json".format(new_speaker_id)
    batch_size = 16
    # exp_dir = 
    exp_dir = os.path.join(exp_base_dir_hifi, exp_dir)
    train_manifest_file = os.path.join(filelist_dir, train_manifest_file)
    val_manifest_file = os.path.join(filelist_dir, val_manifest_file)
    
    training_command = "python examples/tts/hifigan_finetune.py --config-name=hifigan44100.yaml train_dataset={} validation_datasets={} +init_from_ptl_ckpt={} trainer.max_steps={}  model.train_ds.dataloader_params.batch_size={} model.validation_ds.dataloader_params.batch_size={} exp_manager.exp_dir={}  ~model.sched".format(
        train_manifest_file, val_manifest_file, hifipretrained_ckpt, trainer_max_steps, batch_size, batch_size, exp_dir)
    return training_command

In [28]:
overall_command = ""
for mixing in [False, True]:
    for speaker in [6097, 92]:

        for duration_mins in [1, 5, 30, 60]:
            overall_command += generate_hifigan_training_command(speaker, duration_mins, mixing)
            overall_command += ";"
print(overall_command)

python examples/tts/hifigan_finetune.py --config-name=hifigan44100.yaml train_dataset=/home/pneekhara/filelists/6097_mainifest_train_dur_1_mins_local.json validation_datasets=/home/pneekhara/filelists/6097_mainifest_dev_ns_all_local.json +init_from_ptl_ckpt=/home/pneekhara/PreTrainedModels/HiFiMix_No_92_6097.ckpt trainer.max_steps=2000  model.train_ds.dataloader_params.batch_size=16 model.validation_ds.dataloader_params.batch_size=16 exp_manager.exp_dir=/home/pneekhara/ExperimentsHiFiFinetuning/8051_to_6097_no_mixing_1_mins  ~model.sched;python examples/tts/hifigan_finetune.py --config-name=hifigan44100.yaml train_dataset=/home/pneekhara/filelists/6097_mainifest_train_dur_5_mins_local.json validation_datasets=/home/pneekhara/filelists/6097_mainifest_dev_ns_all_local.json +init_from_ptl_ckpt=/home/pneekhara/PreTrainedModels/HiFiMix_No_92_6097.ckpt trainer.max_steps=2000  model.train_ds.dataloader_params.batch_size=16 model.validation_ds.dataloader_params.batch_size=16 exp_manager.exp_di

In [19]:
generate_training_command_new(92, 5, False, 8051, use_new_pitch_stats=True)

'python examples/tts/fastpitch2_finetune.py --config-name=fastpitch_align_44100.yaml train_dataset=/home/pneekhara/filelists/92_mainifest_train_dur_5_mins_local.json validation_datasets=/home/pneekhara/filelists/92_mainifest_dev_ns_all_local.json +init_from_nemo_model=/home/pneekhara/PreTrainedModels/FastPitch.nemo trainer.max_epochs=1000 trainer.check_val_every_n_epoch=1 prior_folder=/home/pneekhara/Datasets/78419/Hi_Fi_TTS_v_0_backup/Priors92 model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 exp_manager.exp_dir=/home/pneekhara/ExperimentsMainBranch/8051_to_92_no_mixing_5_mins model.n_speakers=1 model.pitch_avg=214.5 model.pitch_std=30.9 model.pitch_fmin=80 model.pitch_fmax=512 model.optim.lr=2e-4 ~model.optim.sched'

In [17]:
ctr = 0
overall_command = ""
for mixing in [False, True]:
    for speaker in [6097, 92]:
    
        for duration_mins in [1]:
            overall_command += generate_training_command_new(speaker, duration_mins, mixing, 8051, use_new_pitch_stats=True)
            overall_command += ";"

In [18]:
overall_command

'python examples/tts/fastpitch2_finetune.py --config-name=fastpitch_align_44100.yaml train_dataset=/home/pneekhara/filelists/6097_mainifest_train_dur_1_mins_local.json validation_datasets=/home/pneekhara/filelists/6097_mainifest_dev_ns_all_local.json +init_from_nemo_model=/home/pneekhara/PreTrainedModels/FastPitch.nemo trainer.max_epochs=5000 trainer.check_val_every_n_epoch=1 prior_folder=/home/pneekhara/Datasets/78419/Hi_Fi_TTS_v_0_backup/Priors6097 model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 exp_manager.exp_dir=/home/pneekhara/ExperimentsAutomatedResetPitch/8051_to_6097_no_mixing_1_mins model.n_speakers=1 model.pitch_avg=121.9 model.pitch_std=23.1 model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 ~model.optim.sched;python examples/tts/fastpitch2_finetune.py --config-name=fastpitch_align_44100.yaml train_dataset=/home/pneekhara/filelists/92_mainifest_train_dur_1_mins_local.json validation_datasets=/home/pneekhara/fileli

In [7]:
float("FastPitch--v_loss=1.80-epoch=0.ckpt".split("v_loss=")[1].split("-epoch")[0])

1.8

In [12]:
ctr = 0
overall_command = ""
for speaker in [92, 6097]:
    for duration_mins in [5, 30, 60]:
        for mixing in [False, True]:
            print(get_best_ckpt(speaker, duration_mins, mixing, 8051))
            print("**")

[(1.16, '/home/pneekhara/ExperimentsAutomated/8051_to_92_no_mixing_5_mins/FastPitch/2021-07-20_17-57-18/checkpoints/FastPitch--v_loss=1.16-epoch=37.ckpt'), (1.19, '/home/pneekhara/ExperimentsAutomated/8051_to_92_no_mixing_5_mins/FastPitch/2021-07-20_17-57-18/checkpoints/FastPitch--v_loss=1.19-epoch=199-last.ckpt'), (1.19, '/home/pneekhara/ExperimentsAutomated/8051_to_92_no_mixing_5_mins/FastPitch/2021-07-20_17-57-18/checkpoints/FastPitch--v_loss=1.19-epoch=36.ckpt'), (1.19, '/home/pneekhara/ExperimentsAutomated/8051_to_92_no_mixing_5_mins/FastPitch/2021-07-20_17-57-18/checkpoints/FastPitch--v_loss=1.19-epoch=39.ckpt')]
**
[(2.32, '/home/pneekhara/ExperimentsAutomated/8051_to_92_mixing_5_mins/FastPitch/2021-07-20_18-12-56/checkpoints/FastPitch--v_loss=2.32-epoch=4-last.ckpt'), (2.34, '/home/pneekhara/ExperimentsAutomated/8051_to_92_mixing_5_mins/FastPitch/2021-07-20_18-12-56/checkpoints/FastPitch--v_loss=2.34-epoch=2.ckpt'), (2.44, '/home/pneekhara/ExperimentsAutomated/8051_to_92_mixing

In [ ]:
python examples/tts/fastpitch2_finetune.py --config-name=fastpitch_align_finetuningnomixing train_dataset=filelists/92_mainifest_train_dur_5_mins_local.json validation_datasets=filelists/92_mainifest_dev_ns_all_local.json +init_from_nemo_model=/home/pneekhara/PreTrainedModels/FastPitch.nemo trainer.max_epochs=1000 trainer.check_val_every_n_epoch=1 prior_folder=/home/pneekhara/Datasets/Priors92 model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 exp_manager.exp_dir=/home/pneekhara/ExperimentsAutomatedO/8051_to_92_no_mixing_5_mins